# Instructions:

1) run this notebook to update modules and serve the server.py
2) in cli python notebooks\mcp\langgraph_mcp_ex\main.py 


source: https://gaodalie.substack.com/p/langgraph-mcp-ollama-the-key-to-powerful


# NBA MCP Example with LangGraph and Ollama using deepseek

In [1]:
%%writefile agent.py

# agent.py
from langchain_core.messages import AIMessage, ToolMessage, HumanMessage
from langgraph.graph import StateGraph, START, END, MessagesState
from nodes import create_chatbot
import asyncio
from langchain_mcp_adapters.client import MultiServerMCPClient

async def verify_mcp_connection(url):
    """
    Explicitly verify the MCP connection and available tools.
    """
    try:
        async with MultiServerMCPClient({"server": {"url": url, "transport": "sse", "timeout": 10}}) as client:
            tools = client.get_tools()
            print(f"[MCP CONNECTION] Verified connection to {url}. Available tools: {[tool.name for tool in tools]}")
            return True
    except Exception as e:
        print(f"[MCP ERROR] Could not connect to MCP server at {url}: {e}")
        return False

async def create_agent(docs_info=None):
    server_url = "http://localhost:8000/sse"
    
    # Explicit MCP connection verification
    if not await verify_mcp_connection(server_url):
        raise ConnectionError(f"Cannot connect to MCP at {server_url}")

    async with MultiServerMCPClient({
        "server": {
            "url": server_url,
            "transport": "sse",
            "timeout": 30
        }
    }) as client:
        tools = client.get_tools()
        print(f"[MCP] Connected to {server_url}. Tools fetched: {[tool.name for tool in tools]}")

        graph_builder = StateGraph(MessagesState)

        chatbot_node = create_chatbot(docs_info)
        graph_builder.add_node("chatbot", chatbot_node)

        async def async_tool_executor(state):
            messages = state["messages"]
            last_message = messages[-1]

            tool_calls = getattr(last_message, "tool_calls", None) or \
                         last_message.additional_kwargs.get("tool_calls", None)

            if not tool_calls:
                return {"messages": messages}

            new_messages = messages.copy()
            for tool_call in tool_calls:
                tool_name = tool_call.get("name") if isinstance(tool_call, dict) else tool_call.name
                tool_args = tool_call.get("args", {}) if isinstance(tool_call, dict) else getattr(tool_call, "args", {})
                tool_id = tool_call.get("id", "tool-call-id") if isinstance(tool_call, dict) else getattr(tool_call, "id", "tool-call-id")

                print(f"[TOOL CALL] {tool_name} invoked with arguments: {tool_args}")

                tool = next((t for t in tools if t.name == tool_name), None)
                if not tool:
                    error_msg = f"[TOOL ERROR] Invalid tool '{tool_name}'. Valid tools: {[t.name for t in tools]}"
                    print(error_msg)
                    new_messages.append(AIMessage(content=error_msg))
                    continue

                try:
                    result = await tool.coroutine(**tool_args) if asyncio.iscoroutinefunction(tool.coroutine) else \
                             tool.func(**tool_args) if hasattr(tool, 'func') else tool(**tool_args)
                    print(f"[TOOL RESULT] {tool_name} result: {result}")
                    new_messages.append(ToolMessage(content=str(result), tool_call_id=tool_id, name=tool_name))
                except Exception as e:
                    error_detail = f"[TOOL EXECUTION ERROR] {tool_name} failed: {e}"
                    print(error_detail)
                    new_messages.append(AIMessage(content=error_detail))

            return {"messages": new_messages}

        graph_builder.add_node("tools", async_tool_executor)

        def router(state):
            last_message = state["messages"][-1]
            text = last_message.content.lower() if hasattr(last_message, "content") else ""
            if "career statistics" in text and "nba" in text:
                print("[ROUTER FALLBACK] Detected NBA statistics query. Routing to 'tools'.")
                return "tools"
            has_tool_calls = bool(getattr(last_message, "tool_calls", None) or last_message.additional_kwargs.get("tool_calls"))
            route = "tools" if has_tool_calls else "end"
            print(f"[ROUTER] Routing to '{route}' based on last message.")
            return route


        graph_builder.add_edge(START, "chatbot")
        graph_builder.add_conditional_edges("chatbot", router, {"tools": "tools", "end": END})
        graph_builder.add_edge("tools", "chatbot")

        graph = graph_builder.compile()
        return graph, client




Overwriting agent.py


In [2]:
%%writefile nodes.py

# nodes.py
from server import get_tools
from langgraph.graph import MessagesState
# Removed OpenAI dependency; now using ChatOllama for an open-source LLM.
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from datetime import datetime
import os

# Create a dynamic system prompt that includes the current date and tool list.
def get_system_prompt(docs_info=None):
    system_prompt = f"""
    Today is {datetime.now().strftime("%Y-%m-%d")}
    You are a helpful AI Assistant that can use the following tools:
    - data_visualization: Create charts with Python and matplotlib.
    - python_repl: Execute Python code.
    - nba_player_stats: Retrieve NBA player career statistics (overall, last 5 seasons).
    - search_nba_player: Search for NBA players by full name.
    - nba_player_game_logs: Retrieve game logs for an NBA player for a specific season.
    - nba_team_stats: Retrieve team statistics for a given NBA team and season.
    - nba_player_stats_regularseason: Retrieve regular season career totals.
    - nba_player_stats_postseason: Retrieve postseason career totals.
    - nba_player_stats_career_totals: Retrieve an aggregated career totals summary (regular season only).
    - nba_player_stats_best_season: Retrieve the player's best season based on points per game.
    
    IMPORTANT: When a query asks for NBA career statistics, select the most appropriate tool:
      • Use nba_player_stats for overall career stats.
      • Use nba_player_stats_regularseason for regular season only.
      • Use nba_player_stats_postseason for postseason only.
      • Use nba_player_stats_career_totals for a career totals summary.
      • Use nba_player_stats_best_season to find the best season based on key metrics.
    
    Do not output raw tool responses; simply acknowledge execution and summarize the result.
    When using image generation or data visualization tools, only confirm execution without revealing raw details.
    Once a tool has been executed, do not call it again in the same answer.
    """
    if docs_info:
        docs_context = "\n\nYou have access to these documents:\n"
        for doc in docs_info:
            docs_context += f"- {doc['name']}: {doc['type']}\n"
        system_prompt += docs_context
        
    system_prompt += "\nYou should always answer in the same language as the user's query."
    return system_prompt




# Instantiate the LLM using ChatOllama, configured to use Llama3.2.
# Updated for deepseek-r1
llm = ChatOllama(model="deepseek-r1:latest", temperature=0.7, max_tokens=512)


# Create the chatbot node that processes user input and interacts with the LLM.
def create_chatbot(docs_info=None):
    prompt = ChatPromptTemplate.from_messages([
        SystemMessagePromptTemplate.from_template(get_system_prompt(docs_info)),
        HumanMessagePromptTemplate.from_template("{input}")
    ])
    
    # Pipe the prompt into the LLM chain.
    chain = prompt | llm
    
    def chatbot(state: MessagesState):
        # Normalize messages to the proper format.
        if isinstance(state["messages"], str):
            from langchain_core.messages import HumanMessage
            messages = [HumanMessage(content=state["messages"])]
        else:
            messages = state["messages"]
            
        response = chain.invoke(messages)
        return {"messages": messages + [response]}
    
    return chatbot


Overwriting nodes.py


In [3]:
%%writefile server.py
# server.py
from mcp.server.fastmcp import FastMCP
from langchain_experimental.utilities import PythonREPL
import io
import base64
import matplotlib.pyplot as plt
from pydantic import BaseModel, Field
import asyncio
from nba_api.stats.endpoints import playercareerstats, playergamelog, teamyearbyyearstats
import pandas as pd
from nba_api.stats.static import players, teams

# Create an instance of FastMCP – this is crucial!
mcp = FastMCP("My MCP Server")

# Existing tool: nba_player_stats
@mcp.tool()
def nba_player_stats(player_name: str):
    player_dict = players.find_players_by_full_name(player_name)
    if not player_dict:
        return f"No player found with name '{player_name}'."
    
    player_id = player_dict[0]['id']
    stats = playercareerstats.PlayerCareerStats(player_id=player_id)
    df = stats.get_data_frames()[0]
    summary = df[['SEASON_ID', 'TEAM_ABBREVIATION', 'GP', 'PTS', 'REB', 'AST']].tail(5).to_string(index=False)
    return f"Career stats for {player_name} (last 5 seasons):\n{summary}"

# New Tool: search_nba_player
@mcp.tool()
def search_nba_player(query: str) -> str:
    from nba_api.stats.static import players
    results = players.find_players_by_full_name(query)
    if not results:
        return f"No NBA players found for query '{query}'."
    info = f"Found {len(results)} players for '{query}':\n"
    for p in results:
        info += f"ID: {p['id']} - Name: {p['full_name']} - Team: {p.get('teamName', 'N/A')}\n"
    return info

# New Tool: nba_player_game_logs
@mcp.tool()
async def nba_player_game_logs(player_name: str, season: str) -> str:
    from nba_api.stats.static import players
    from nba_api.stats.endpoints import playergamelog
    results = players.find_players_by_full_name(player_name)
    if not results:
        return f"No player found with name '{player_name}'."
    player_id = results[0]['id']
    game_logs = playergamelog.PlayerGameLog(player_id=player_id, season=season)
    df = game_logs.get_data_frames()[0]
    summary = df[['GAME_DATE', 'MATCHUP', 'PTS', 'REB', 'AST']].to_string(index=False)
    return f"Game logs for {player_name} in {season}:\n{summary}"

# New Tool: nba_team_stats
@mcp.tool()
def nba_team_stats(team_abbreviation: str, season: str) -> str:
    from nba_api.stats.static import teams
    from nba_api.stats.endpoints import teamyearbyyearstats
    team_list = teams.find_teams_by_abbreviation(team_abbreviation)
    if not team_list:
        return f"No team found with abbreviation '{team_abbreviation}'."
    team_id = team_list[0]['id']
    stats = teamyearbyyearstats.TeamYearByYearStats(team_id=team_id)
    df = stats.get_data_frames()[0]
    df_season = df[df['YEAR'] == season] if season in df['YEAR'].values else df
    summary = df_season.to_string(index=False)
    return f"Team stats for {team_abbreviation} in {season}:\n{summary}"

@mcp.tool()
def nba_player_stats_regularseason(player_name: str) -> str:
    """
    Retrieve the NBA player's career regular season totals.
    Uses the NBA API's PlayerCareerStats endpoint and returns only the Regular Season data.
    """
    player_dict = players.find_players_by_full_name(player_name)
    if not player_dict:
        return f"No player found with name '{player_name}'."
    
    player_id = player_dict[0]['id']
    stats = playercareerstats.PlayerCareerStats(player_id=player_id)
    try:
        df_reg = stats.get_data_frames()[0]
        summary = df_reg[['SEASON_ID', 'TEAM_ABBREVIATION', 'GP', 'PTS', 'REB', 'AST']].tail(5).to_string(index=False)
        return f"Regular Season Stats for {player_name} (last 5 seasons):\n{summary}"
    except Exception as e:
        return f"Error retrieving regular season stats for {player_name}: {str(e)}"

@mcp.tool()
def nba_player_stats_postseason(player_name: str) -> str:
    """
    Retrieve the NBA player's career postseason totals.
    Uses the NBA API's PlayerCareerStats endpoint and returns only the Postseason data.
    """
    player_dict = players.find_players_by_full_name(player_name)
    if not player_dict:
        return f"No player found with name '{player_name}'."
    
    player_id = player_dict[0]['id']
    stats = playercareerstats.PlayerCareerStats(player_id=player_id)
    try:
        # Assuming the postseason totals are in the second DataFrame (index 1)
        df_post = stats.get_data_frames()[1]
        if df_post.empty:
            return f"No postseason stats available for {player_name}."
        summary = df_post[['SEASON_ID', 'TEAM_ABBREVIATION', 'GP', 'PTS', 'REB', 'AST']].tail(5).to_string(index=False)
        return f"Postseason Stats for {player_name} (last 5 seasons):\n{summary}"
    except Exception as e:
        return f"Error retrieving postseason stats for {player_name}: {str(e)}"

@mcp.tool()
def nba_player_stats_career_totals(player_name: str) -> str:
    """
    Retrieve an aggregated summary of the NBA player's career totals (combining regular season data).
    This tool returns overall totals and per-game averages based on regular season data.
    """
    player_dict = players.find_players_by_full_name(player_name)
    if not player_dict:
        return f"No player found with name '{player_name}'."
    
    player_id = player_dict[0]['id']
    stats = playercareerstats.PlayerCareerStats(player_id=player_id)
    try:
        df_reg = stats.get_data_frames()[0]
        if df_reg.empty:
            return f"No regular season data available for {player_name}."
        
        totals = df_reg[['GP', 'PTS', 'REB', 'AST']].sum()
        averages = df_reg[['PTS', 'REB', 'AST']].mean()
        summary = (
            f"Career Totals for {player_name} (Regular Season):\n"
            f"Games Played: {totals['GP']}\n"
            f"Total Points: {totals['PTS']}\n"
            f"Total Rebounds: {totals['REB']}\n"
            f"Total Assists: {totals['AST']}\n\n"
            f"Average Points per Game: {averages['PTS']:.2f}\n"
            f"Average Rebounds per Game: {averages['REB']:.2f}\n"
            f"Average Assists per Game: {averages['AST']:.2f}"
        )
        return summary
    except Exception as e:
        return f"Error retrieving career totals for {player_name}: {str(e)}"
    

# Existing tools for other functions remain unchanged
repl = PythonREPL()


@mcp.tool()
def data_visualization(code: str):
    try:
        repl.run(code)
        buf = io.BytesIO()
        plt.savefig(buf, format='png')
        buf.seek(0)
        img_str = base64.b64encode(buf.getvalue()).decode()
        return f"data:image/png;base64,{img_str}"
    except Exception as e:
        return f"Error creating chart: {str(e)}"
        
@mcp.tool()
def nba_player_stats_for_season(player_name: str, season: str) -> str:
    player_dict = players.find_players_by_full_name(player_name)
    if not player_dict:
        return f"No player found with name '{player_name}'."
    player_id = player_dict[0]['id']
    stats = playercareerstats.PlayerCareerStats(player_id=player_id)
    df = stats.get_data_frames()[0]
    if season not in df['SEASON_ID'].values:
        return f"Data for season {season} is not available."
    df_season = df[df['SEASON_ID'] == season]
    summary = df_season[['SEASON_ID', 'TEAM_ABBREVIATION', 'GP', 'PTS', 'REB', 'AST']].to_string(index=False)
    return f"Stats for {player_name} in {season}:\n{summary}"

@mcp.tool()
def nba_player_avg_ppg_for_season(player_name: str, season: str) -> str:
    # Find the player by full name.
    player_dict = players.find_players_by_full_name(player_name)
    if not player_dict:
        return f"No player found with name '{player_name}'."
    
    player_id = player_dict[0]['id']
    stats = playercareerstats.PlayerCareerStats(player_id=player_id)
    df = stats.get_data_frames()[0]
    
    # Check if the season is present in the data.
    if season not in df['SEASON_ID'].values:
        return f"Data for season {season} is not available."
    
    # Filter the DataFrame to only include the requested season.
    df_season = df[df['SEASON_ID'] == season]
    
    # Calculate the average points per game.
    try:
        avg_ppg = df_season['PTS'].mean()
        return f"{avg_ppg:.2f}"
    except Exception as e:
        return f"Error computing average points for {player_name} in {season}: {str(e)}"

@mcp.tool()
def python_repl(code: str):
    return repl.run(code)

def get_tools(retriever_tool=None):
    base_tools = [
        python_repl, 
        data_visualization, 
        nba_player_stats, 
        search_nba_player, 
        nba_player_game_logs, 
        nba_team_stats,
        nba_player_stats_for_season,
        nba_player_avg_ppg_for_season
    ]
    if retriever_tool:
        base_tools.append(retriever_tool)
    return base_tools

if __name__ == "__main__":
    mcp.run(transport="sse")


Overwriting server.py


In [4]:
%%writefile main.py

# main.py
import streamlit as st
import asyncio
from agent import create_agent
from langchain_core.messages import HumanMessage

async def main():
    # Create the agent and keep the MCP client alive.
    agent, client = await create_agent()
    
    # Get user input (using input() for a console example)
    user_input = input("What would you like to ask? ")
    initial_message = HumanMessage(content=user_input)
    
    try:
        print("Processing your request...")
        result = await agent.ainvoke({"messages": [initial_message]})
        
        # Iterate over returned messages and print them.
        for message in result["messages"]:
            if hasattr(message, "type") and message.type == "human":
                print(f"User: {message.content}")
            elif hasattr(message, "type") and message.type == "tool":
                print(f"Tool Result: {message.content}")
                if "image" in message.content.lower() and "url" in message.content.lower():
                    print("Image Generated Successfully!")
            else:
                print(f"AI: {message.content}")
    except Exception as e:
        print(f"Error: {str(e)}")
    
    # In a real application, the client would remain active as long as needed.

if __name__ == "__main__":
    asyncio.run(main())


Overwriting main.py


In [5]:
# New Cell: Start MCP Server in the Background

import threading
import asyncio
from server import mcp  # Ensure this imports the FastMCP instance from your server.py

def start_mcp_server():
    # Run the MCP server asynchronously using the SSE transport.
    asyncio.run(mcp.run(transport="sse"))

# Start the MCP server in a background daemon thread.
server_thread = threading.Thread(target=start_mcp_server, daemon=True)
server_thread.start()

print("MCP server started in the background.")


MCP server started in the background.


In [6]:
import socket
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
try:
    s.bind(("0.0.0.0", 8000))
    print("Port 8000 is free.")
except Exception as e:
    print("Port 8000 is in use:", e)
finally:
    s.close()


Port 8000 is free.


In [7]:
import nest_asyncio
import asyncio
from agent import create_agent
from langchain_core.messages import HumanMessage

# Allow nested event loops in the notebook
nest_asyncio.apply()

async def run_agent():
    agent, client = await create_agent()
    user_query = input("What would you like to ask? ")
    initial_message = HumanMessage(content=user_query)
    result = await agent.ainvoke({"messages": [initial_message]})
    for message in result["messages"]:
        print(message.content)

# Run the agent asynchronously
await run_agent()


[MCP CONNECTION] Verified connection to http://localhost:8000/sse. Available tools: ['nba_player_stats', 'search_nba_player', 'nba_player_game_logs', 'nba_team_stats', 'nba_player_stats_regularseason', 'nba_player_stats_postseason', 'nba_player_stats_career_totals', 'data_visualization', 'nba_player_stats_for_season', 'nba_player_avg_ppg_for_season', 'python_repl']
[MCP] Connected to http://localhost:8000/sse. Tools fetched: ['nba_player_stats', 'search_nba_player', 'nba_player_game_logs', 'nba_team_stats', 'nba_player_stats_regularseason', 'nba_player_stats_postseason', 'nba_player_stats_career_totals', 'data_visualization', 'nba_player_stats_for_season', 'nba_player_avg_ppg_for_season', 'python_repl']
[ROUTER FALLBACK] Detected NBA statistics query. Routing to 'tools'.
[ROUTER FALLBACK] Detected NBA statistics query. Routing to 'tools'.
[ROUTER FALLBACK] Detected NBA statistics query. Routing to 'tools'.
[ROUTER FALLBACK] Detected NBA statistics query. Routing to 'tools'.
[ROUTER] Ro